This is a simple natural language dataframe question-answering system, which is based on the LangChain framework and combines OllamaLLM's Llama 3.1 LLM, Chroma vector database and HuggingFace's all-MiniLM-L6-v2 embedding model. These allow the system to be lightweight, localized and flexible.



Environment preparation

These are the packages and modules needed to run this system. I have already installed them and will not run them here.

```bash
pip install pandas langchain langchain-ollama langchain-chroma langchain-huggingface
```

Import required Python libraries and modules

In [1]:

import pandas as pd 
from langchain_chroma import Chroma 
from langchain.schema import Document
from langchain_ollama import OllamaLLM
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain_huggingface import HuggingFaceEmbeddings



- `pd`: DataFrame operations and data analysis
- `Chroma`: Vector database for storing and searching embeddings
- `Document`: Container for text content and metadata
- `OllamaLLM`: Interface for local LLM models
- `PromptTemplate`: Template builder for LLM inputs
- `StrOutputParser`: Converts LLM outputs to strings
- `RunnablePassthrough`:Pass the user's question directly to subsequent processing steps
- `HuggingFaceEmbeddings`: A bridge to make Hugging Face’s embedding capabilities easier to use in the LangChain environment

### part 1: Data preparation
#### step1: Load data

Load csv data, the file is downloaded from kaggle: https://www.kaggle.com/datasets/spscientist/students-performance-in-exams

Only 20 rows of data are randomly selected to facilitate observation of model effects

In [2]:
df = pd.read_csv('random_sample.csv') 
df.shape

(20, 8)

In [3]:
df

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group C,associate's degree,standard,none,91,86,84
1,female,group B,some college,free/reduced,completed,53,66,73
2,male,group D,bachelor's degree,standard,none,80,73,72
3,male,group C,some college,free/reduced,none,74,77,73
4,male,group E,some college,standard,completed,84,83,78
5,male,group D,associate's degree,free/reduced,none,81,75,78
6,male,group B,associate's degree,free/reduced,completed,69,70,63
7,female,group B,some high school,standard,completed,54,61,62
8,male,group C,associate's degree,free/reduced,none,87,73,72
9,male,group B,some high school,standard,completed,51,54,41


#### step2: store the column name information in the vector database as embeddings 

Dataframe is structured data. Each row of data has the same structure and column name. If all dataframe information is stored in the vector database, the retrieval effect will be reduced due to a large amount of identical structural information.

To solve this, the key task of this RAG system is not to directly return the dataframe query result to the user. Instead, it converts the user's question into corresponding Pandas query code, and Python executes the code to get the final result.

For generating Pandas code, only the column name information is needed. So, we only store the column names in the vector database to allow efficient similarity search later.

1. Store column name information in a structured document object for embedding in the next step


In [4]:

col_info = f"The column names of the dataset are: {df.columns.tolist()}."

documents = [Document(page_content=col_info, metadata={"type": "column_info"})]

documents 

[Document(metadata={'type': 'column_info'}, page_content="The column names of the dataset are: ['gender', 'race/ethnicity', 'parental level of education', 'lunch', 'test preparation course', 'math score', 'reading score', 'writing score'].")]

2. Embedding document content and storing it in the vector database

This is done to enable efficient semantic search, so the system can quickly find documents related to the user's question.




In [5]:
embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embedding
    )

all-MiniLM-L6-v2 is a lightweight and fast embedding model, which is suitable for this simple RAG system.

chroma is a efficient, flexible and easy to use vector database.

### part 2: Create a RAG question-answering system

#### Step 1: Create a Retriever

A vector database is a database that stores embedding vectors. It also works as a retriever to find the embedded documents most similar to the user's question.So, we use the vectorstore create a retriever.

In [6]:
#k: Amount of documents to return, default is 4, there is only 1 document in my document, so set k=1
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

#### Step 2: Create an Augmentor

The augmentor is used to improve the retriever's performance. It combines the retrieved document content, the user's question, and the rules for generating pandas code. This input is sent to the LLM, helping it better understand the user's intent and generate more accurate pandas query code.

Because our vector database stores very little information (only column names), setting up the augment information is very important. I created a clear and detailed prompt for this purpose here.


In [ ]:
template = """You are a dataframe analysis assistant. Provide concise answers with only 1 section:

               Code: [write one line of pandas code in ```python``` block, 
                        EXAMPLE:code:
                                ```python
                                    df['category_name'].value_counts().count()
                                ```
                        ]


                Context:{context}
                Question: {question}
                
                Note: Use the existing DataFrame 'df' provided by the system, do not create or read a new one.
                
                You can use these pandas operations:
                1. Basic statistics: df.describe(), df[column].mean(), df[column].max(), etc.
                2. Group statistics: df.groupby(column).agg()
                3. Sorting: df.sort_values(by=column)
                4. Filtering: df[df[column] > value]
                ...
                
                Rules:
                1. No explanations or additional text.
                2. write complete executable code in ```python``` block using the existing 'df'.
                3. Use proper column names from the DataFrame.

                """

In [8]:
PROMPT = PromptTemplate(
    template=template,
    input_variables=["context", "question"]  
)

#### step3: Create a Generator

Now, use llm to generate pandas query code based on the retrieved and augmented information

In [9]:
llm = OllamaLLM(model="llama3.1", temperature=0.75)

We can run llama3.1 locally

temperature=0.75, can keep the generated content somewhat creative while not being too random.

#### step4: Create a chain

Use pipeline operations to connect the retriever, augmentor, and generator in series to form a complete rag question-answering system


In [10]:
chain = (
        {"context": retriever,"question": RunnablePassthrough()}
        | PROMPT
        | llm
        | StrOutputParser()
    )

### part3: Result execution

Execute the pandas code returned by rag locally and return the result


In [11]:
def extract_code_and_execute(answer): 
       
    # Extract code block
    code_start = answer.find("```python") + 9
    code_end = answer.find("```", code_start)
    code = answer[code_start:code_end].strip()
    
    # Create local namespace and execute code
    local_dict = {'df': df, 'pd': pd}
    code = f"result = {code}"
    exec(code, None, local_dict)
 
    return local_dict.get('result')

In [12]:
def query_dataframe(question: str):
    
    code = chain.invoke(question)
    result = extract_code_and_execute(code)
    
    return {'code': code, 'result': result}


In [13]:
query = "What is the highest math score?"
query_dataframe(query)

{'code': "```python\ndf['math score'].max()\n```", 'result': 91}

In [14]:
query = "What is the lowest reading score?"
query_dataframe(query)

{'code': "```python\ndf['reading score'].min()\n```", 'result': 26}

In [15]:
df['reading score'].min()

26

In [16]:
df['math score'].max()

91

In [18]:
query = "What are the two lowest reading scores?"
query_dataframe(query)

{'code': "```python\ndf.nsmallest(2, 'reading score')\n```",
 'result':    gender race/ethnicity parental level of education         lunch  \
 11   male        group E            some high school      standard   
 17   male        group D           bachelor's degree  free/reduced   
 
    test preparation course  math score  reading score  writing score  
 11                    none          30             26             22  
 17                    none          55             46             44  }

In [19]:
df.nsmallest(2, 'reading score')

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
11,male,group E,some high school,standard,none,30,26,22
17,male,group D,bachelor's degree,free/reduced,none,55,46,44


In [21]:
df.lunch.unique()

array(['standard', 'free/reduced'], dtype=object)